In [ ]:
# Transformaciones realizadas:
# 1. Renombrar columnas para mantener consistencia (e.g., 'name' → 'company_name').
# 2. Ajuste de longitudes de strings para cumplir con los límites de las columnas en la BD.
# 3. Conversión de tipos de datos:
#    - 'amount' a float y luego filtrado por el máximo permitido en DECIMAL(16,2)
#    - 'created_at' y 'paid_at' a datetime, con NaT convertidos a None para que se guarden como NULL
# 4. Eliminación de filas incompletas (por ejemplo, si 'id', 'company_id', 'amount', 'status' o 'created_at' eran nulos)
# 5. Manejo de valores atípicos y datos corruptos (como montos extremadamente grandes)
#
# Retos encontrados:
# - Fechas en formatos mixtos (yyyy-mm-dd, dd/mm/yyyy, y yyyyMMdd) que requerían parsing personalizado.
# - Los valores nulos en fechas que Pandas convierte a NaT y no se guardaban como NULL en la BD.
# - Valores numéricos extremadamente grandes que excedían el límite de DECIMAL(16,2), requiriendo filtrado.
# - Evitar que campos vacíos de texto se conviertan en la cadena 'None' al guardar en la base de datos.



import pandas as pd
from conexion import get_db_connection
# Se carga de la tabla cruda los datos

df = pd.read_csv("raw_data_extracted.csv")  # Ajusta la ruta si es necesario

In [ ]:
# Vemos una vista de nuestros datos
df.head()

,id,name,company_id,amount,status,created_at,paid_at
0,48ba4bdbfb56ceebb32f2bd0263e759be942af3d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,05fc6f5ac66b6ee7e4253aa5d0c2299eb47aaaf4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,2cdce231c1fc6a2061bfa2f1d978351fe217245d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05


In [ ]:
# emos el tipo de dato de cada columna
df.dtypes

id             object
name           object
company_id     object
amount        float64
status         object
created_at     object
paid_at        object
dtype: object

In [ ]:
# Vemos el número de filas y columnas de nuestro dataset
df.shape

(10000, 7)

In [ ]:
# Vemos cuantas filas por columnas tienen valores nulos
df.isna().sum()

id               3
name             3
company_id       4
amount           0
status           0
created_at       0
paid_at       3991
dtype: int64

In [ ]:
# Renombramos el nombre de la columna para que coincida con el esquema dado
df.rename(columns={'name': 'company_name'}, inplace=True)

In [1052]:
# Eliminar filas con nulos en columnas donde no se admiten nulos en el esquema
df = df.dropna(subset=['id', 'company_id', 'amount', 'status', 'created_at'])


In [1053]:
# cols = ["company_name", "paid_at"]  # columnas que quieres limpiar
# for col in cols:
#     df[col] = df[col].where(df[col].notna(), None)


In [ ]:
# Vemos el total de filas y columnas despues de borrar las filas donde las columnas no permiten nulos (Se perdieron 7 filas)
df.shape

(9993, 7)

In [ ]:
# Vemos el nombre de las columnas
df.columns

Index(['id', 'company_name', 'company_id', 'amount', 'status', 'created_at',
       'paid_at'],
      dtype='object')

## Se inicia la tranformacion de los datos en el formato del esquema

In [ ]:
# Ajustar longitudes de strings al esquema
df['id'] = df['id'].astype(str).str[:24]
df['company_id'] = df['company_id'].astype(str).str[:24]

# Como company_name tiene valores nulos y en el esquema se aceptan, los transformamos a none
df['company_name'] = df['company_name'].where(df['company_name'].notna(), None)

# Despues los transformamos a string con una longitud maxima de 130
df['company_name'] = df['company_name'].astype(str).str[:130] 

# Como los datos nulos se transforron a none y luego estos en cadena 'None', remplazamos los None por tipo None para que posgrest los guarde como nulo
df['company_name'] = df['company_name'].replace('None', None)

# Ajustar longitudes de strings al esquema
df['status'] = df['status'].astype(str).str[:30]




In [ ]:
# Convertir 'amount' a decimal
# Existen valores que superan los 16 digitos permitidos por el esquema
max_amount = 99999999999999.99  # límite de DECIMAL(16,2)
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')  # convierte strings a números
df = df[df['amount'] <= max_amount]  # elimina filas con valores demasiado grandes
# df['amount'] = pd.to_numeric(df['amount'].round(2))

In [1058]:
# Convertir fechas
df['paid_at'] = pd.to_datetime(df['paid_at'], errors='coerce')  # Asigna NaT donde falla
df['paid_at'] = df['paid_at'].astype(object)                     # cambia a object
df['paid_at'] = df['paid_at'].where(pd.notna(df['paid_at']), None)  # reemplaza NaT por None
df

,id,company_name,company_id,amount,status,created_at,paid_at
0,48ba4bdbfb56ceebb32f2bd0,MiPasajefy,cbf1c8b09cd5b549416d49d2,3.00,voided,2019-03-19,None
1,05fc6f5ac66b6ee7e4253aa5,MiPasajefy,cbf1c8b09cd5b549416d49d2,3.00,pending_payment,2019-05-06,None
2,2cdce231c1fc6a2061bfa2f1,MiPasajefy,cbf1c8b09cd5b549416d49d2,3.00,voided,2019-02-22,None
3,81633ba310a50b673efd469c,MiPasajefy,cbf1c8b09cd5b549416d49d2,102.61,paid,2019-02-27,2019-02-27 00:00:00
4,6ccfc4c24e788e4bca448df3,MiPasajefy,cbf1c8b09cd5b549416d49d2,184.49,paid,2019-02-05,2019-02-05 00:00:00
...,...,...,...,...,...,...,...
9995,4907c36c037bfa4874047e14,MiPasajefy,cbf1c8b09cd5b549416d49d2,59.88,paid,2019-03-14,2019-03-14 00:00:00
9996,ef9ec85887418d2ccf220686,MiPasajefy,cbf1c8b09cd5b549416d49d2,23.05,paid,2019-02-20,2019-02-20 00:00:00
9997,50bb3950ecde94ca3e078530,MiPasajefy,cbf1c8b09cd5b549416d49d2,20.50,paid,2019-03-19,2019-03-19 00:00:00
9998,1b2c4e503a4a952064b70369,MiPasajefy,cbf1c8b09cd5b549416d49d2,81.42,paid,2019-03-10,2019-03-10 00:00:00


In [ ]:
# Revisamos que los valores nulos esten entipo de dato None para que se guarden en la base de datos como nulos
df['paid_at'].unique()

array([None, Timestamp('2019-02-27 00:00:00'),
       Timestamp('2019-02-05 00:00:00'), Timestamp('2019-01-22 00:00:00'),
       Timestamp('2019-01-12 00:00:00'), Timestamp('2019-03-29 00:00:00'),
       Timestamp('2019-05-18 00:00:00'), Timestamp('2019-04-16 00:00:00'),
       Timestamp('2019-03-04 00:00:00'), Timestamp('2019-04-03 00:00:00'),
       Timestamp('2019-05-10 00:00:00'), Timestamp('2019-01-02 00:00:00'),
       Timestamp('2019-04-11 00:00:00'), Timestamp('2019-03-01 00:00:00'),
       Timestamp('2019-03-10 00:00:00'), Timestamp('2019-02-07 00:00:00'),
       Timestamp('2019-03-15 00:00:00'), Timestamp('2019-04-29 00:00:00'),
       Timestamp('2019-04-02 00:00:00'), Timestamp('2019-03-20 00:00:00'),
       Timestamp('2019-05-15 00:00:00'), Timestamp('2019-04-04 00:00:00'),
       Timestamp('2019-01-08 00:00:00'), Timestamp('2019-03-18 00:00:00'),
       Timestamp('2019-03-08 00:00:00'), Timestamp('2019-04-24 00:00:00'),
       Timestamp('2019-02-18 00:00:00'), Timestamp('2

In [ ]:
# Observamos que la columna created_at tienen distintos formatos las fechas
df['created_at'].unique()

array(['2019-03-19', '2019-05-06', '2019-02-22', '2019-02-27',
       '2019-02-05', '2019-01-04', '2019-05-04', '2019-01-23',
       '2019-05-03', '2019-01-22', '2019-03-02', '2019-01-12',
       '2019-03-29', '2019-05-18', '2019-01-10', '2019-01-11',
       '2019-02-16', '2019-04-16', '2019-05-10', '2019-01-03',
       '2019-03-04', '2019-04-03', '2019-01-15', '2019-01-02',
       '2019-04-11', '2019-03-01', '2019-03-08', '2019-03-10',
       '2019-02-07', '2019-03-14', '2019-03-15', '2019-03-22',
       '2019-04-29', '2019-05-02', '2019-05-12', '2019-04-02',
       '2019-05-05', '2019-04-12', '2019-01-16', '2019-03-20',
       '2019-05-15', '2019-05-09', '2019-04-04', '2019-01-08',
       '2019-03-18', '2019-04-08', '2019-04-24', '2019-02-18',
       '2019-03-28', '2019-05-19', '2019-02-24', '2019-01-31',
       '2019-04-25', '2019-02-13', '2019-01-05', '2019-02-12',
       '2019-03-23', '2019-01-24', '2019-02-21', '2019-02-19',
       '2019-03-12', '2019-03-27', '2019-02-14', '2019-

In [ ]:
# Creamos una función para convertirlos las fechas que tienen otro formato al de la mayoría

def parse_fecha(x):
    try:
        # Principal: dd/mm/yyyy
        return pd.to_datetime(x)
    except:
        # Formatos específicos sin dayfirst
        for fmt in ("%Y-%m-%dT%H:%M:%S", "%Y%m%d"):
            try:
                return pd.to_datetime(x, format=fmt)
            except:
                continue
        return pd.NaT

df['created_at'] = df['created_at'].apply(parse_fecha)



In [1062]:
# # Convertir strings a datetime, NaT donde no se pueda
# df['paid_at'] = pd.to_datetime(df['paid_at'])

# # Reemplazar NaT por None y asegurar tipo object
# df['paid_at'] = df['paid_at'].apply(lambda x: x if pd.notna(x) else None).astype(object)

# type(df.iloc[0]['paid_at'])


In [ ]:
# Confirmamos que los valores nulos en company_name esten como tipo de dato None para que se guarden nulos en la base de datos
df['company_name'].unique()

array(['MiPasajefy', 'Muebles chidos', None, 'MiPas0xFFFF', 'MiP0xFFFF'],
      dtype=object)

In [ ]:
# Revisamos cuantos valores nulos nos quedan por columna y confirmamos las que no deben ser nulos por no permitirlo el esquema
# Observamos que solo tenemnos valores nulos en las columnas que si lo permite el esquema
df.isna().sum()

id                 0
company_name       3
company_id         0
amount             0
status             0
created_at         0
paid_at         3984
dtype: int64

In [ ]:
# Se guardan en csv los datos_transformados que cumplen con el esquema dado en las instrucciones de la prueba

df.to_csv('transformed_data.csv')

In [ ]:
# Se crea la tabla conforme al esquema y se confirma que los datos se puedan cargar


conn = get_db_connection()
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS transformed_data (
    id VARCHAR(24) NOT NULL,
    company_name VARCHAR(130) NULL,
    company_id VARCHAR(24) NOT NULL,
    amount DECIMAL(16,2) NOT NULL,
    status VARCHAR(30) NOT NULL,
    created_at TIMESTAMP NOT NULL,
    paid_at TIMESTAMP NULL
)
""")
conn.commit()
print("✅ Tabla 'transformed_data' creada")

# df = df.where(pd.notnull(df), None)
# df = df.where(pd.notna(df), None)
# df['paid_at'] = df['paid_at'].where(pd.notna(df['paid_at']), None)

for row in df.itertuples(index=False):
    cur.execute("""
        INSERT INTO transformed_data (id, company_name, company_id, amount, status, created_at, paid_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """,  row)

conn.commit()
cur.close()
conn.close()
print("✅ CSV cargado en la tabla 'transformed_dta'")

✅ Tabla 'transformed_data' creada
✅ CSV cargado en la tabla 'transformed_dta'
